Libraries

In [ ]:
import os
from datetime import datetime
import pandas as pd
import re
from pypdf import PdfReader
import datefinder
from rapidfuzz import process
from dateutil import parser as date_parser

Load Vendor Data

In [ ]:
EXCEL_PATH = r"PATH_TO_YOUR_EXCEL_FILE.xlsx" # Replace with the actual path to your Excel file
rm_df = pd.read_excel(EXCEL_PATH, sheet_name="Sheet Name", header=1) # Enter actual sheet name, change header to 0 if the first row is the header

Shared Utility Functions. Make sure the parameter names line up with how you format the excel file:

In [3]:
def get_initials(full_name):
    return ''.join([part[0].upper() for part in str(full_name).split() if part])

def get_rm_initials(company_name): 
    row = rm_df[rm_df['Vendor Name'].str.lower() == company_name.lower()]
    if not row.empty:
        rm_name = row.iloc[0]['Relationship Manager']
        return get_initials(rm_name)
    return 'XX'

def get_last_modified_date(filepath):
    timestamp = os.path.getmtime(filepath)
    return datetime.fromtimestamp(timestamp).strftime('%Y%m%d')

def detect_doc_type(filename):  # if loops are in order of specificity to generality
    name = filename.lower()
    if "a" in name or "b" in name:
        return "c"
    elif "d" in name or "e" in name:
        return "f"
    # ... Add more specific checks as needed
    else:
        return "g" # Default case

def guess_vendor_from_filename_or_ird(filename, vendor_list, ird_list):
    base = os.path.splitext(os.path.basename(filename))[0]
    # Regex for ID: optional apostrophe, 4 digits, optional letter, optional apostrophe, e.g. '5097', '5097A', 5097A, etc.
    id_match = re.search(r"'?\s*(\d{4}[A-Za-z]?)\s*'?", base)
    if id_match:
        ird_candidate = id_match.group(1)
        # Try to match ID in DataFrame (case-insensitive, strip spaces)
        row = rm_df[rm_df['Id'].astype(str).str.replace("'", "").str.strip().str.lower() == ird_candidate.lower()]
        if not row.empty:
            return row.iloc[0]['Vendor Name']
    # Fallback: vendor name guessing
    words = re.split(r'[\s\-_\.]+', base)
    ignore = {'words'} # Add common legal/document type words that might skew guessing
    vendor_guess = ' '.join([w for w in words if w.lower() not in ignore and not w.isdigit() and not re.match(r'\d{4}', w)])
    match = process.extractOne(vendor_guess, vendor_list, score_cutoff=60)
    return match[0] if match else None

Draft Naming Functions

In [ ]:
def build_draft_filename_auto(filepath):
    vendor_names = rm_df['Vendor Name'].tolist()
    ird_list = rm_df['Id'].astype(str).tolist()
    guessed_vendor = guess_vendor_from_filename_or_ird(os.path.basename(filepath), vendor_names, ird_list)
    if guessed_vendor is None:
        guessed_vendor = "UnknownVendor"
    draft_date = get_last_modified_date(filepath)
    doc_type = detect_doc_type(os.path.basename(filepath))
    rm_initials = get_rm_initials(guessed_vendor)
    return f"{draft_date}-My Company Name-{guessed_vendor}-{doc_type}-{rm_initials}" # Alter this format as needed.
filepath = r"" #path to your file you'd like to rename
new_name = build_draft_filename_auto(filepath)
print(new_name)

20240909-ISO-UnknownVendor-VSAL-XX


Executed (PDF) Naming Functions

In [ ]:
#Extracting potential date strings with more specific formats
def extract_date_strings(text):
    patterns = [
        # ISO format dates (most reliable)
        r'\b\d{4}[-/]\d{1,2}[-/]\d{1,2}\b',
        # US format MM/DD/YYYY
        r'\b\d{1,2}/\d{1,2}/\d{4}\b',
        # Written dates like "January 12, 2025"
        r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4}\b',
        # Abbreviated months "Jan 12, 2025"
        r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.?\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4}\b',
        # Day first format "12 January 2025"
        r'\b\d{1,2}(?:st|nd|rd|th)?\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b',
    ]

    all_dates = []
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        all_dates.extend(matches)
    return all_dates

#Parse dates and filter out obviously wrong ones
def parse_and_validate_dates(date_strings):
    valid_dates = []
    current_year = datetime.now().year

    for date_str in date_strings:
        try:
            parsed_date = date_parser.parse(date_str, fuzzy=False)
            if 1990 <= parsed_date.year <= current_year + 10:
                valid_dates.append(parsed_date)
        except (ValueError, TypeError):
            continue
    return valid_dates

# Look for patterns like "effective as of", "is effective", etc.
def find_effective_date_context(text):
    effective_patterns = [
        r'effective\s+(?:as\s+of\s+)?([^.]{0,50}(?:\d{1,2}[/-]\d{1,2}[/-]\d{4}|\w+\s+\d{1,2},?\s+\d{4})[^.]{0,20})',
        r'(?:shall\s+be\s+)?effective\s+(?:on\s+)?([^.]{0,50}(?:\d{1,2}[/-]\d{1,2}[/-]\d{4}|\w+\s+\d{1,2},?\s+\d{4})[^.]{0,20})',
        r'(?:agreement|contract|document)\s+(?:is\s+)?effective\s+([^.]{0,50}(?:\d{1,2}[/-]\d{1,2}[/-]\d{4}|\w+\s+\d{1,2},?\s+\d{4})[^.]{0,20})'
    ]

    for pattern in effective_patterns:
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            context = match.group(1)
            dates = extract_date_strings(context)
            if dates:
                parsed = parse_and_validate_dates(dates)
                if parsed:
                    return max(parsed)  # Return latest date in effective context
    return None

#Big kahuna
def select_pdf_date_for_naming(filepath):
    try:
        doc_type = detect_doc_type(os.path.basename(filepath))
        with open(filepath, 'rb') as f:
            reader = PdfReader(f)
            first_page_text = ""
            last_page_text = ""
            if len(reader.pages) > 0:
                first_page_text = reader.pages[0].extract_text() or ""
            if len(reader.pages) > 1:
                last_page_text = reader.pages[-1].extract_text() or ""
            else:
                last_page_text = first_page_text

            # Clean up extracted text
            first_page_clean = re.sub(r'\s+', ' ', first_page_text.strip())
            last_page_clean = re.sub(r'\s+', ' ', last_page_text.strip())
            # Debugging output, helps to know where it pulled the date from without opening the PDF
            print(f"Debug - Doc type: {doc_type}")
            print(f"Debug - First 200 chars: {first_page_clean[:200]}")

            # Priority 1: For Annex documents, prioritize first page
            if doc_type and doc_type.startswith("Annex"):
                dates = parse_and_validate_dates(extract_date_strings(first_page_clean))
                if dates:
                    selected_date = min(dates)  # Use earliest date for Annex
                    print(f"Debug - Annex date selected: {selected_date}")
                    return selected_date.strftime('%Y%m%d')
        
            # Priority 2: Look for effective date context
            effective_date = find_effective_date_context(first_page_clean + " " + last_page_clean)
            if effective_date:
                print(f"Debug - Found effective date: {effective_date}")
                return effective_date.strftime('%Y%m%d')

            # Priority 3: Look for signature dates (usually on last page)
            signature_context = re.search(r'(?:signed|executed|dated).*?([^.]{0,100})', last_page_clean, re.IGNORECASE)
            if signature_context:
                sig_dates = parse_and_validate_dates(extract_date_strings(signature_context.group(1)))
                if sig_dates:
                    selected_date = max(sig_dates)
                    print(f"Debug - Signature date selected: {selected_date}")
                    return selected_date.strftime('%Y%m%d')

            # If all else fails, fallback to all dates, prefer later ones
            all_dates = parse_and_validate_dates(
                extract_date_strings(first_page_clean) + 
                extract_date_strings(last_page_clean)
            )
            if all_dates:
                unique_dates = list(set(all_dates))
                unique_dates.sort()
                selected_date = max(unique_dates)
                print(f"Debug - Fallback date selected: {selected_date}")
                return selected_date.strftime('%Y%m%d')
            # If no valid dates found, return "Review", debug lets me know the failure was in finding a date
            print("Debug - No valid dates found")
            return "Review"
     # and if any error occurs, return "Review" to avoid breaking the process
    except Exception as e:
        print(f"Error processing PDF {filepath}: {str(e)}")
        return "Review"

def build_executed_filename_auto(filepath):
    """Build filename for executed documents"""
    vendor_names = rm_df['Vendor Name'].tolist()
    ird_list = rm_df['Ird Id'].astype(str).tolist()
    guessed_vendor = guess_vendor_from_filename_or_ird(
        os.path.basename(filepath), vendor_names, ird_list
    )
    if guessed_vendor is None:
        guessed_vendor = "UnknownVendor"
    eff_date = select_pdf_date_for_naming(filepath)
    doc_type = detect_doc_type(os.path.basename(filepath))
    return f"{eff_date}-ISO-{guessed_vendor}-{doc_type}"

pdf_path = r"" #Path to your PDF file
new_pdf_name = build_executed_filename_auto(pdf_path)
print(f"Generated filename: {new_pdf_name}")

Debug - Doc type: VSAL
Debug - First 200 chars: Ed. 12/2023 1 VERISK STRATEGIC ALLIANCES LICENSE AGREEMENT, including all Annexes attached hereto (the "Agreement "), dated August 27, 2024, between Insurance Services Office, Inc., with its principal
Debug - Fallback date selected: 2024-08-27 00:00:00
Generated filename: 20240827-ISO-UnknownVendor-VSAL
